Importing libraries

In [3]:
import os
import glob
import pandas as pd 
import csv
import collections
import pprint



Identification and quantification of data

In [80]:
data_directory = '../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Initial data/'
raw_files = os.listdir(data_directory)

path, dirs, files = next(os.walk(data_directory))
file_count = len(files)

fso = com.Dispatch("Scripting.FileSystemObject")
folder = fso.GetFolder(data_directory)
MB = 1024 * 1024.0
print("There are",file_count,".csv file. The total size of the data set are : %.2f MB" % (folder.Size / MB))
print("Files:") 
raw_files

There are 24 .csv file. The total size of the data set are : 175.22 MB
Files:


['201803_demo.it_201905139_2232439_powerelectricity.csv',
 '201803_demo.it_201905139_2302655_Presence.csv',
 '201803_demo.it_201905139_230269_ReedSensor.csv',
 '201803_demo.it_201905140_0012800_LuxMeter.csv',
 '201804_demo.it_201905139_2241977_powerelectricity.csv',
 '201804_demo.it_201905139_2302312_Presence.csv',
 '201804_demo.it_201905139_2303629_ReedSensor.csv',
 '201804_demo.it_201905140_0013825_LuxMeter.csv',
 '201805_demo.it_201905139_2241968_powerelectricity.csv',
 '201805_demo.it_201905139_2302581_Presence.csv',
 '201805_demo.it_201905139_2302679_ReedSensor.csv',
 '201805_demo.it_201905140_0013843_LuxMeter.csv',
 '201902_demo.it_201905139_1610271_LuxMeter.csv',
 '201902_demo.it_201905139_1612442_Presence.csv',
 '201902_demo.it_201905139_1710863_ReedSensor.csv',
 '201902_demo.it_201905139_1716752_Environmental.csv',
 '201903_demo.it_201905139_1443721_LuxMeter.csv',
 '201903_demo.it_201905139_1615869_Presence.csv',
 '201903_demo.it_201905139_1706199_ReedSensor.csv',
 '201903_dem

Divide the .csv files according to the type of sensor

In [65]:
powerelectricity = sorted([f for f in raw_files if "powerelectricity" in f])
powerelectricity_len = len(powerelectricity)

presence = sorted([f for f in raw_files if "Presence" in f])
presence_len = len(presence)

reedsensor = sorted([f for f in raw_files if "ReedSensor" in f])
reedsensor_len = len(reedsensor)

luxmeter = sorted([f for f in raw_files if "LuxMeter" in f])
luxmeter_len = len(luxmeter)

environmental = sorted([f for f in raw_files if "Environmental" in f])
environmental_len = len(environmental)

totalcsvnum = powerelectricity_len + presence_len + reedsensor_len + luxmeter_len +environmental_len

print(  "The number of Powerelectricty .csv files are:",powerelectricity_len,  f'\nPowerelectricty: {powerelectricity}  \
        \n\nThe number of Presence .csv files are: {presence_len} \nPresence: {presence} \
        \n\nThe number of Reedsensor .csv files are: {reedsensor_len} \nReedsensor: {reedsensor} \
        \n\nThe number of Luxmeter .csv files are: {luxmeter_len} \nLuxmeter: {luxmeter} \
        \n\nThe number of Environmental .csv files are: {environmental_len} \nEnvironmental: {environmental}\
        \n\nThe total number of divided csv is:{totalcsvnum} ')

sensors  = {'reedsensor':reedsensor, 'luxmeter':luxmeter, 'environmental':environmental, 'presence':presence, 'powerelectricity':powerelectricity}

The number of Powerelectricty .csv files are: 3 
Powerelectricty: ['201803_demo.it_201905139_2232439_powerelectricity.csv', '201804_demo.it_201905139_2241977_powerelectricity.csv', '201805_demo.it_201905139_2241968_powerelectricity.csv']          

The number of Presence .csv files are: 6 
Presence: ['201803_demo.it_201905139_2302655_Presence.csv', '201804_demo.it_201905139_2302312_Presence.csv', '201805_demo.it_201905139_2302581_Presence.csv', '201902_demo.it_201905139_1612442_Presence.csv', '201903_demo.it_201905139_1615869_Presence.csv', '201904_demo.it_201905139_1615548_Presence.csv']         

The number of Reedsensor .csv files are: 6 
Reedsensor: ['201803_demo.it_201905139_230269_ReedSensor.csv', '201804_demo.it_201905139_2303629_ReedSensor.csv', '201805_demo.it_201905139_2302679_ReedSensor.csv', '201902_demo.it_201905139_1710863_ReedSensor.csv', '201903_demo.it_201905139_1706199_ReedSensor.csv', '201904_demo.it_201905139_1705203_ReedSensor.csv']         

The number of Luxmeter

Create a new folder for the new processed files

In [82]:
Newdata_directory = '../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/'

try:
    os.mkdir(Newdata_directory)
except OSError:
    print ("Creation of the directory %s failed" % Newdata_directory)
else:
    print ("Successfully created the directory %s " % Newdata_directory)

Creation of the directory ../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/ failed


Adding the .csv files combined based on the type of sensor in the directory created

In [95]:
for sensor_name, sensor_files in sensors.items():
    df = pd.concat([pd.read_csv(data_directory + f, sep=';') for f in sensor_files])
    df.drop(df.columns[[-1,]], axis=1, inplace=True)
    df.to_csv(Newdata_directory + sensor_name + '.csv', index=False)
    print(f'Merged {sensor_name}.csv files')

Merged reedsensor.csv files
Merged luxmeter.csv files
Merged environmental.csv files
Merged presence.csv files
Merged powerelectricity.csv files


The files luxmeter.csv and presence.csv have data for both 2018 and 2019, 
So we need to split the files by year and delete the file that contains both years

In [48]:
luxmeter_path = '../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter.csv'
presence_path = '../Ivan Compagnucci/EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence.csv'

rowL2019 = pd.read_csv(luxmeter_path, skiprows=range(0,192272)) 
rowL2019.to_csv('EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter2019.csv', index=False, header=None)
print(f'Created the luxmeter2019.csv file')

rowL2018 = pd.read_csv(luxmeter_path, skiprows=range(192272, 447274))
rowL2018.to_csv('EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter2018.csv', index=False, header=None)
print(f'Created the luxmeter2018.csv file')
os.remove('EDA-EnterpriseDataAnalytics-project/Data/Merged data/luxmeter.csv')
print(f'Deleted the luxmeter.csv file')

rowP2018 = pd.read_csv(presence_path, skiprows=range(0, 440500))
rowP2018.to_csv('EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2019.csv', index=False, header=None)
print(f'Created the presence2019.csv file')

rowP2019 = pd.read_csv(presence_path, skiprows=range(440500, 1230379)) 
rowP2019.to_csv('EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence2018.csv', index=False, header=None)
print(f'Created the presence2018.csv file')
os.remove('EDA-EnterpriseDataAnalytics-project/Data/Merged data/presence.csv')
print(f'Deleted the presence.csv file')

Created the luxmeter2019.csv file
Created the luxmeter2018.csv file
Deleted the luxmeter.csv file
Created the presence2019.csv file
Created the presence2018.csv file
Deleted the presence.csv file
